# Euclidean Rhythms

Euclidean Rhythms were discovered in 2004 and first described in [this paper](http://cgm.cs.mcgill.ca/~godfried/publications/banff.pdf)
First we're going to implement the bjorklund algorithm, during which we'll see how these sequences work and hopefully get some intuition as for why they are attractive musically. 
If you don't care about the algorithm and just want to see its application in music21, jump ahead to 
[this section](#application)

### Intuition 
Bjorklund sequences are binary sequences characteriezd by spacing that is as even as possible. 

This is a Bjorklund sequence:

0 1 0 1 0 1 1

This is not:

0 0 0 1 1 1 1

Individual Bjorklund sequences can be defined by three parameters:

- the number of total elements in the sequence - *steps*
- the number of "on" elements in the sequence - *pulses*
- an optional third parameter - *rotation*

Rotation is a little difficult to visualize when using linear sequential notation, but imagine a circular sequence like a clock where every third hour was "on". Expressed linearly, it would look like this:

1 0 0 1 0 0 1 0 0 1 0 0

And if you rotated all the "on"s one hour clockwise, it would look like this:

0 1 0 0 1 0 0 1 0 0 1 0

It's an identical sequence with a different rotation. Linearly, this is expressed by severing some of the end and tacking it onto the beginning. 

This is very musically relevant because the "same" sequences will sound very different rotated. In ethnomusicology, rhythms that are rotations of each other are said to be of the same "rhythmic necklace"





## The Algorithm

## Bringing in Music21
<a id="application">